In [37]:
import pandas as pd
import os

In [38]:
os.chdir("/Users/ayatohisanaga/University of Washington/2024 Spring/IMT 542/project")

In [39]:
traffic_directory = "outputs/bicycle_traffic"
traffic_datasets = [file for file in os.listdir(traffic_directory) if file.endswith('.csv')]

traffic_data = {}

for dataset in traffic_datasets:
    # Extract dataset name (filename without extension)
    dataset_name = os.path.splitext(dataset)[0]
    # Read the CSV file and store the DataFrame in the dictionary
    df = pd.read_csv(os.path.join(traffic_directory, dataset))
    # Convert date column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    # Extract only the date component
    df['date'] = df['date'].dt.date
    # Store the modified DataFrame in the dictionary
    traffic_data[dataset_name] = df

In [40]:
weather_data = pd.read_json("outputs/weather/weather_data.json")
weather_data['date'] = pd.to_datetime(weather_data['date']).dt.date
weather_data.head()

,date,temp_min,temp_max,temp_avg,precipitation,wind_speed_avg
0,2012-12-03,45.9,49.6,48.1,0.00,8.98
1,2012-12-04,46.8,53.2,49.9,0.00,7.83
2,2012-12-05,43.2,50.7,47.1,0.00,10.01
3,2012-12-06,44.4,47.7,45.2,0.00,8.86
4,2012-12-07,43.2,46.9,45.2,0.00,7.13


In [41]:
merged_data = {}

for dataset in traffic_datasets:
    dataset_name = os.path.splitext(dataset)[0]
    df = pd.read_csv(os.path.join(traffic_directory, dataset))
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    # Group by date and sum the traffic counts
    daily_traffic = df.groupby('date').sum().reset_index()
    
    # Merge with weather data
    merged_df = pd.merge(daily_traffic, weather_data, on='date', how='left')
    
    merged_data[dataset_name] = merged_df

In [43]:
output_directory = 'website/datasets'

# Merge and save datasets
traffic_datasets = [f for f in os.listdir(traffic_directory) if f.endswith('.csv')]
for dataset in traffic_datasets:
    dataset_name = os.path.splitext(dataset)[0]
    df = pd.read_csv(os.path.join(traffic_directory, dataset))
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    # Group by date and sum the traffic counts
    daily_traffic = df.groupby('date').sum().reset_index()
    
    # Merge with weather data
    merged_df = pd.merge(daily_traffic, weather_data, on='date', how='left')
    
    # Save merged dataset
    merged_df.to_csv(os.path.join(output_directory, f'{dataset_name}_merged.csv'), index=False)
